# Embeddings - Explicación y Experimento 

Este notebook reproduce las partes centrales del Capítulo 2 (Raschka)
y añade explicaciones en primera persona, con analogías cotidianas,
y un experimento que varía `max_length` y `stride` para mostrar el número
de muestras generadas y por qué el solapamiento es útil.

In [ ]:
# Recomendado: ejecutar en un entorno con torch y tiktoken instalados.
# Si no los tienes, descomenta e instala:
# !pip install --upgrade pip
# !pip install torch tiktoken requests

import sys
import importlib
import subprocess

def ensure_package(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, 
, 
, 
, pkg])

print('Asegúrate de tener `torch` y `tiktoken` instalados antes de ejecutar completamente este notebook.')

## Tokenización y vocabulario (explicación personal)

Imaginemos que el texto es una gran biblioteca y los tokens son los libros individuales.
La tokenización decide qué es un 
: ¿una palabra completa, un fragmento de palabra,
o incluso una letra? Esta decisión afecta lo que el modelo 
 y cómo generaliza.

En términos prácticos, la tokenización determina la granularidad del idioma:
- Un token demasiado grande (p. ej. frases completas) hará que sea difícil aprender
  relaciones porque hay demasiadas unidades distintas.
- Un token demasiado pequeño (p. ej. letras) requiere secuencias muy largas.

Por eso los tokenizers tipo BPE (usado por `tiktoken`) mezclan lo mejor: usan subpalabras
para manejar palabras nuevas sin inflar el vocabulario.

In [ ]:
# Tokenizar con tiktoken (BPE / GPT-2)
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

sample = raw_text[:1000]
ids = tokenizer.encode(sample)
print('Tokens sample length:', len(ids))
print('Primeros 20 token ids:', ids[:20])
print('Decoded (reconstruido):', tokenizer.decode(ids[:50]))

## De tokens a IDs y embeddings (explicación personal con analogía)

Ahora imaginemos que cada token tiene una ficha con un número (ID). Esa ficha es lo que
llevamos a la máquina. Pero las máquinas no entienden fichas numeradas: necesitan vectores,
es decir, una lista de características numéricas. La capa de `Embedding` es como una mesa
donde cada ficha (ID) te da una tarjeta con información (vector).

así: si los tokens son personas, la embedding es su tarjeta de presentación
con rasgos (edad, oficio, intereses) que facilitan encontrar afinidad entre ellas.
Cuanto más parecidas sean las tarjetas, más cerca estarán en el espacio vectorial.

Así lo entiendo yo.

In [ ]:
# Crear dataset tipo sliding-window (adaptado del libro)
import torch
from torch.utils.data import Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # tokenizar todo el texto con tiktoken (ya es BPE)
        token_ids = tokenizer.encode(txt, allowed_special={
})
        assert len(token_ids) > max_length, 'tokenized length must be > max_length'
        self.input_ids = []
        self.target_ids = []
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunk, dtype=torch.long))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

## Sliding window y solapamiento (explicación con ejemplo diario)

Imagínemos que queremos aprender una canción larga memorizando fragmentos. Si memorizamos
segmentos que se solapan (repitimos la última frase del fragmento anterior), te será más fácil
reconstruir la canción completa porque recuerdas la unión entre fragmentos.

En ML, ese solapamiento (stride pequeño) genera más ejemplos y ayuda al modelo a ver cómo
las fronteras entre fragmentos encajan. Pero hay un trade-off: demasiada repetición =
menos información nueva por ejemplo y riesgo de sobreajuste.

In [ ]:
# Experimento: variar max_length y stride y reportar número de samples
params = [(128,128),(128,64),(128,32),(256,256),(256,128),(256,64)]
results = []
for max_length,stride in params:
    ds = GPTDatasetV1(raw_text, tokenizer, max_length=max_length, stride=stride)
    results.append((max_length,stride,len(ds)))

print('max_length, stride -> num_samples')
for r in results:
    print(r)

# Observación: mostramos también la cantidad total de tokens y la fórmula esperada aproximada:
total_tokens = len(tokenizer.encode(raw_text))
print('
Total tokens (approx):', total_tokens)

# Una explicación rápida automatizada por cada experimento:
for max_length,stride,n in results:
    possible_positions = max(0, total_tokens - max_length)
    expected = (possible_positions // stride)
    print(f'For max_length={max_length:3}, stride={stride:3} -> samples={n:6} (approx expected={expected})')

## Resultado del experimento (explicación personal)

- `max_length` define cuánto contexto ve el modelo en cada ejemplo (como cuánto de la canción recuerdas).
- `stride` controla cuánto avanzamos entre ejemplos; `stride < max_length` implica solapamiento.

Analogía: si `max_length` es el tamaño de tu hoja de estudio y `stride` cuánto avanzamos al pasar la hoja,
un `stride` pequeño significa que volvemos a leer partes que ya vimos (refuerzo), mientras que un `stride` grande
significa leer contenido mayormente nuevo (mayor diversidad).

## ¿Por qué las embeddings codifican significado? 

Voy a explicarlo como si yo se lo contara a un compañero:

1) Hipótesis distribucional (intuitiva):
- He notado que palabras que aparecen en contextos similares (por ejemplo, "perro" y "gato") tienden
  a estar relacionadas; las embeddings capturan esa idea transformando co-ocurrencias en distancias.

2) Analogía de la vida diaria:
- Piensa en una fiesta donde cada persona recibe una tarjeta con varios rasgos: "le gusta el jazz", "es vegetariano",
  "trabaja en software". Las personas con tarjetas parecidas tienden a agruparse. Una embedding es exactamente esa tarjeta:
  un vector de rasgos que facilita encontrar afinidades.

3) Relación con redes neuronales:
- Una embedding es simplemente una fila de la matriz de pesos W en la capa `Embedding`.
  Cuando entrenas la red por backprop, estas filas (vectores) cambian para que las proximidades en el espacio
  reflejen relaciones útiles para la tarea.

4) Contextualización:
- Una embedding estática (por token) captura información general; sin embargo, los modelos modernos generan
  embeddings contextuales (la misma palabra tiene vectores distintos según la frase). Estos últimos codifican significado
  dinámico y son más precisos para tareas complejas.

5) Limitaciones y sentido práctico: 
- Embeddings no son magia: capturan correlaciones del dato de entrenamiento. Si el corpus está sesgado, las embeddings
  también lo estarán. Además, la dimensión (p. ej. 256 vs 1024) es un trade-off entre capacidad y coste.

In [ ]:
# Mostrar creación de embeddings token -> vector y su forma
import torch
vocab_size = 50257  # GPT-2 BPE vocab size (tiktoken)
output_dim = 256
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# Tomemos algunos token ids de ejemplo y los embedemos
example_ids = torch.tensor(ids[:8], dtype=torch.long)
embs = embedding_layer(example_ids)
print('Example ids:', example_ids.tolist())
print('Embeddings shape:', embs.shape)